"""
Task 4 — Portfolio Optimization using MPT
- TSLA expected return from forecast (Task 3)
- BND/SPY expected returns from historical averages (annualized)
- Covariance from historical daily returns (TSLA/BND/SPY)
- Efficient Frontier (Monte Carlo), Max Sharpe & Min Volatility
- Outputs plot + CSV + summary report

Required inputs:
- CSV with historical OHLCV for TSLA, BND, SPY (must contain 'Date' and 'Adj Close')
- CSV with TSLA forecast from Task 3 (columns: Forecast, Lower, Upper; index as dates)
"""

In [ ]:
#import  dependences
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from datetime import datetime
import sys
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

In [ ]:
# -----------------------------
# 0) Config
# -----------------------------
CONFIG = {
    # Paths to historical CSVs (from Task 1)
    "tsla_hist_csv": "data/scrap data/TSLA_data.csv",
    "bnd_hist_csv":  "data/scrap data/BND_data.csv",
    "spy_hist_csv":  "data/scrap data/SPY_data.csv",

    # Path to TSLA forecast results (from Task 3)
    "tsla_forecast_csv": "data/output/tesla_forecast_6months.csv",  # columns: Forecast, Lower, Upper

    # Columns
    "date_col": "Date",
    "price_col": "Adj Close",  # fallback to 'Close' if not present

    # Output
    "output_dir": "data/output",
    "frontier_png": "efficient_frontier.png",
    "frontier_csv": "efficient_frontier_samples.csv",
    "summary_json": "portfolio_summary.json",
    "summary_txt": "portfolio_summary.txt",

    # Simulation
    "n_portfolios": 50000,
    "risk_free_rate": 0.02,  # annualized
    "trading_days": 252
}